In [1]:
import pandas as pd
import numpy as np
import statistics as st

In [19]:
data = pd.read_csv("IMDB Dataset.csv")

In [20]:
data = data.iloc[:10000]

In [21]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [22]:
data["review"][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [23]:
data["sentiment"].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [24]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [25]:
data.duplicated().sum()

17

In [26]:
data.drop_duplicates(inplace=True)

In [27]:
data.duplicated().sum()

0

In [28]:
# BAsic preprossing
# remove html tags
#Lower casing of letters
#removeing stopwords

In [29]:
from bs4 import BeautifulSoup
def remove_html_tags(input_string):
    soup = BeautifulSoup(input_string, "html.parser")
    cleaned_text = soup.get_text()
    return cleaned_text

In [30]:
data["review"] = data["review"].apply(remove_html_tags)

C:\Users\yash mohite\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [31]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
9995,"Fun, entertaining movie about WWII German spy ...",positive
9996,Give me a break. How can anyone say that this ...,negative
9997,This movie is a bad movie. But after watching ...,negative
9998,This is a movie that was probably made to ente...,negative


In [34]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

def remove_stop_words(input_text, language='english'):
    stop_words = set(stopwords.words(language))
    words = word_tokenize(input_text)
    filtered_words = ' '.join(word for word in words if word.lower() not in stop_words)
    return filtered_words

[nltk_data] Downloading package stopwords to C:\Users\yash
[nltk_data]     mohite\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\yash
[nltk_data]     mohite\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
data["review"] = data["review"].apply(remove_stop_words)

In [37]:
data["review"] = data["review"].apply(lambda x:x.lower())

In [38]:
data

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production . filming techniqu...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically 's family little boy ( jake ) thinks...,negative
4,petter mattei 's `` love time money '' visuall...,positive
...,...,...
9995,"fun , entertaining movie wwii german spy ( jul...",positive
9996,give break . anyone say `` good hockey movie '...,negative
9997,movie bad movie . watching endless series bad ...,negative
9998,"movie probably made entertain middle school , ...",negative


In [40]:
x = data.drop("sentiment",axis=1)
y = data["sentiment"]

In [41]:
x

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production . filming techniqu...
2,thought wonderful way spend time hot summer we...
3,basically 's family little boy ( jake ) thinks...
4,petter mattei 's `` love time money '' visuall...
...,...
9995,"fun , entertaining movie wwii german spy ( jul..."
9996,give break . anyone say `` good hockey movie '...
9997,movie bad movie . watching endless series bad ...
9998,"movie probably made entertain middle school , ..."


In [42]:
y

0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object

In [43]:
from sklearn.preprocessing import LabelEncoder
encoding = LabelEncoder()
y = encoding.fit_transform(y)

In [44]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [45]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [46]:
X_train.shape

(7986, 1)

In [47]:
#apply Bag Of Word
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [48]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [52]:
X_train_bow.shape

(7986, 48235)

In [49]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [54]:
# Train the model
nb.fit(X_train_bow,y_train)

MultinomialNB()

In [55]:
y_pred = nb.predict(X_test_bow)

In [56]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.85678517776665

In [57]:
confusion_matrix(y_test,y_pred)

array([[863, 122],
       [164, 848]], dtype=int64)

In [58]:
# apply random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8527791687531296

In [60]:
confusion_matrix(y_test,y_pred)

array([[839, 146],
       [148, 864]], dtype=int64)

In [62]:
# using N-Grams
cv = CountVectorizer(ngram_range=(1,5),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()
nb.fit(X_train_bow,y_train)
y_pred = nb.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8668002003004507

In [64]:
# Using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])
nb.fit(X_train_tfidf,y_train)
y_pred = nb.predict(X_test_tfidf)
accuracy_score(y_test,y_pred)

0.8683024536805208

In [65]:
# Using Word2Vec
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [66]:
story = []
for doc in data['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [67]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [68]:
model.build_vocab(story)

In [69]:
model.train(story,total_examples=model.corpus_count,epochs=1)

(1132682, 1198926)

In [70]:
len(model.wv.index_to_key)

31829

In [71]:
def document_vector(doc):
    #remove out of vocabulery words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc],axis=0)

In [79]:
document_vector(data["review"].values[1])

array([-0.17145403,  0.46029449,  0.22381602,  0.24453337,  0.10469473,
       -0.88037807,  0.05863564,  1.0877705 , -0.38471743, -0.36741644,
       -0.43120912, -0.75957733,  0.00810174,  0.22245564, -0.07510801,
       -0.2355994 ,  0.1734267 , -0.4825689 ,  0.14749333, -1.2877315 ,
        0.44695023,  0.2493383 ,  0.40802875, -0.4150496 , -0.07623144,
        0.04489565, -0.4445292 , -0.27636203, -0.5713864 ,  0.13123639,
        0.33795726,  0.23512039,  0.04528222, -0.58235973,  0.00690824,
        0.39640138,  0.09368875, -0.39169663, -0.12086163, -0.7905653 ,
        0.21689862, -0.40041247, -0.0700064 , -0.02730151,  0.57400227,
       -0.18303843, -0.5145302 , -0.04483129,  0.21382847,  0.44004968,
        0.3488726 , -0.29137373, -0.34653842, -0.07684106, -0.24961075,
        0.47335252,  0.33662763,  0.02719381, -0.66236526,  0.2319796 ,
        0.3488812 ,  0.23124707, -0.03472349, -0.0235358 , -0.6793431 ,
        0.38394406,  0.14327495,  0.15883756, -0.61793524,  0.74

In [80]:
from tqdm import tqdm
x = []
for doc in tqdm(data["review"].values):
    x.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [08:04<00:00, 20.61it/s]


In [103]:
x = np.array(x)

In [104]:
x.shape

(9983, 100)

In [105]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [106]:
X_train.shape

(7986, 100)

In [107]:
y_test

array([0, 0, 1, ..., 1, 0, 0])

In [108]:
nb = MultinomialNB()
nb.fit(X_train,y_train)
y_pred = nb.predict(X_test)
accuracy_score(y_test,y_pred)

ValueError: Negative values in data passed to MultinomialNB (input X)